In [ ]:
# from google.colab import drive
# import pandas as pd
# from statistics import mean,median
# import numpy as np
# import matplotlib.pyplot as plt
# import random
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# import networkx as nx
# from networkx.algorithms import bipartite

# Data Preprocessing

In [ ]:
users = pd.read_csv("/content/drive/MyDrive/main/ml-1m/ml_1m_csv/movielens_users.csv", encoding='ISO-8859-1',header=None)       # UTF-8 incompatible with movies.csv
movies = pd.read_csv("/content/drive/MyDrive/main/ml-1m/ml_1m_csv/movielens_movies.csv", encoding='ISO-8859-1')
ratings = pd.read_csv("/content/drive/MyDrive/main/ml-1m/ml_1m_csv/movielens_ratings.csv", encoding='ISO-8859-1', header=None)

### User data

In [ ]:
users.columns = ['user_id','gender','age_group','occupation','zip']
users.head()

,user_id,gender,age_group,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,2460
4,5,M,25,20,55455


In [ ]:
# user id
u = []
for i in range(len(users)):
  u.append('u' + str(users['user_id'][i]))
users['user_id'] = u
users.head()

,user_id,gender,age_group,occupation,zip
0,u1,F,1,10,48067
1,u2,M,56,16,70072
2,u3,M,25,15,55117
3,u4,M,45,7,2460
4,u5,M,25,20,55455


In [ ]:
# zip
for i in range(len(users)):
  zip = str(users['zip'][i])
  if len(zip)>5:
    zip = list(zip.split('-'))[0]
  users['zip'][i] = int(zip)
users.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,gender,age_group,occupation,zip
0,u1,F,1,10,48067
1,u2,M,56,16,70072
2,u3,M,25,15,55117
3,u4,M,45,7,2460
4,u5,M,25,20,55455


In [ ]:
# zip - outliers - 
count = 0
for i in range(len(users)):
  zip = str(users['zip'][i])
  if len(zip)>5:
    print(users["user_id"][i], end=" ")
    print(zip)
    count +=1
    # zip = list(zip.split('-'))[0]
  # users['zip'][i] = int(zip)
count

u541 5849574
u1091 345567
u1259 2020010
u1434 495321
u2106 495321
u2853 444555
u3355 400060
u3905 361069
u4454 111225
u4538 956456
u4913 970025
u4973 949702
u5100 193122042
u5510 191004
u5904 954025


15

In [ ]:
print("Minimum zip:",min(users['zip']))
print("Maximum zip:",max(users['zip']))
print("Mean zip:",mean(users['zip']))
print("Median zip:",median(users['zip']))

Minimum zip: 231
Maximum zip: 193122042
Mean zip: 87590.82930463576
Median zip: 55107.0


In [ ]:
# handle zip outliers - assign median

median = int(median(users["zip"]))
for i in range(len(users)):
  zip = str(users['zip'][i])
  if len(zip)>5:
    users["zip"][i] = median
    print(users["user_id"][i], end=" ")
    print(users["zip"][i])

users.head()

u541 55107
u1091 55107
u1259 55107
u1434 55107
u2106 55107
u2853 55107
u3355 55107
u3905 55107
u4454 55107
u4538 55107
u4913 55107
u4973 55107
u5100 55107
u5510 55107
u5904 55107


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,user_id,age_group,occupation,zip,M,F,gender_switch,occupation_one_hot,age_group_one_hot
0,u1,1,10,48067,0,1,0,0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0
1,u2,56,16,70072,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0,0 0 0 0 0 0 1
2,u3,25,15,55117,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0,0 0 1 0 0 0 0
3,u4,45,7,2460,1,0,1,0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 1 0 0
4,u5,25,20,55455,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 1 0 0 0 0


In [ ]:
# gender

gender = LabelEncoder().fit_transform(users['gender'])
gender = OneHotEncoder().fit_transform(gender.reshape(-1, 1)).toarray()
gender_F = gender[:,0].astype(int)
gender_M = gender[:,1].astype(int)

users = users.drop('gender',axis = 1)
users['M'] = gender_M
users['F'] = gender_F
users['gender_switch']=users['M'].astype(int)
users.head()

,user_id,age_group,occupation,zip,M,F,gender_switch
0,u1,1,10,48067,0,1,0
1,u2,56,16,70072,1,0,1
2,u3,25,15,55117,1,0,1
3,u4,45,7,2460,1,0,1
4,u5,25,20,55455,1,0,1


In [ ]:
# occupation

occu = LabelEncoder().fit_transform(users['occupation'])
occu = OneHotEncoder().fit_transform(occu.reshape(-1, 1)).toarray()
occu_str = []
for i in range(len(occu)):
  occu_str.append(str(occu[i].astype(int)).lstrip("[").rstrip("]"))
users["occupation_one_hot"] = occu_str
users.head()

,user_id,age_group,occupation,zip,M,F,gender_switch,occupation_one_hot
0,u1,1,10,48067,0,1,0,0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
1,u2,56,16,70072,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
2,u3,25,15,55117,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
3,u4,45,7,2460,1,0,1,0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
4,u5,25,20,55455,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


In [ ]:
# age_group

age = LabelEncoder().fit_transform(users['age_group'])
age = OneHotEncoder().fit_transform(age.reshape(-1, 1)).toarray()
age_group = []
for i in range(len(age)):
  age_group.append(str(age[i].astype(int)).lstrip("[").rstrip("]"))
users["age_group_one_hot"] = age_group
users.head()

,user_id,age_group,occupation,zip,M,F,gender_switch,occupation_one_hot,age_group_one_hot
0,u1,1,10,48067,0,1,0,0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0,1 0 0 0 0 0 0
1,u2,56,16,70072,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0,0 0 0 0 0 0 1
2,u3,25,15,55117,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0,0 0 1 0 0 0 0
3,u4,45,7,2460,1,0,1,0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0,0 0 0 0 1 0 0
4,u5,25,20,55455,1,0,1,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1,0 0 1 0 0 0 0


In [ ]:
users.head()
# users.to_csv("drive/My Drive/main/ml-1m/users_processed.csv")

In [ ]:
USER_OCCUPATIONS = ["other" ,"academic/educator", "artist","clerical/admin","college/grad student","customer service","doctor/health care","executive/managerial","farmer","homemaker","K-12 student","lawyer","programmer","retired","sales/marketing","scientist","self-employed","technician/engineer","tradesman/craftsman","unemployed","writer"]
USER_AGE_GROUP = {1:  "Under 18",	18:  "18-24",	25:  "25-34",	35:  "35-44",	45:  "45-49",	50:  "50-55",	56:  "56+"}

### Movies Data

In [ ]:
movies.head()

,id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# movie_id
m = []
for i in range(len(movies)):
  m.append('m' + str(movies['id'][i]))
movies['id'] = m
movies.head()

,id,title,genre
0,m1,Toy Story (1995),Animation|Children's|Comedy
1,m2,Jumanji (1995),Adventure|Children's|Fantasy
2,m3,Grumpier Old Men (1995),Comedy|Romance
3,m4,Waiting to Exhale (1995),Comedy|Drama
4,m5,Father of the Bride Part II (1995),Comedy


In [ ]:
#title
titles = []
years = []
for i in range(len(movies)):
  t = movies["title"][i].rstrip()
  title = t[:-7]
  year = t[-5:-1]
  titles.append(title)
  years.append(year)
movies = movies.drop("title", axis=1)
movies["title"] = titles
movies["year"] = years
movies.head()

,id,genre,title,year
0,m1,Animation|Children's|Comedy,Toy Story,1995
1,m2,Adventure|Children's|Fantasy,Jumanji,1995
2,m3,Comedy|Romance,Grumpier Old Men,1995
3,m4,Comedy|Drama,Waiting to Exhale,1995
4,m5,Comedy,Father of the Bride Part II,1995


In [ ]:
MOVIE_GENRES = ['Action','Adventure','Animation',"Children's",'Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']

In [ ]:
#genre
gen_bins = []
for i in range(len(movies)):
  bin = ['0' for _ in range(18)]
  genres = list(movies['genre'][i].split('|'))
  for j in genres:
    bin[MOVIE_GENRES.index(j)]='1'
  gen_bins.append(' '.join(bin))

movies = movies.drop('genre',axis=1)
movies["genre"] = gen_bins
movies.head()

,id,title,year,genre
0,m1,Toy Story,1995,0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
1,m2,Jumanji,1995,0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
2,m3,Grumpier Old Men,1995,0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0
3,m4,Waiting to Exhale,1995,0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
4,m5,Father of the Bride Part II,1995,0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
# movies.to_csv("drive/My Drive/main/ml-1m/movies_processed.csv")

### Ratings


In [ ]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.columns = ['user_id','movie_id','rating','timestamp']

In [ ]:
u_r, m_r = [], []
for i in range(len(ratings)):
  u_r.append('u' + str(ratings['user_id'][i]))
  m_r.append('m' + str(ratings['movie_id'][i]))
ratings['user_id'] = u_r
ratings['movie_id'] = m_r
ratings.head()

,user_id,movie_id,rating,timestamp
0,u1,m1193,5,978300760
1,u1,m661,3,978302109
2,u1,m914,3,978301968
3,u1,m3408,4,978300275
4,u1,m2355,5,978824291


In [ ]:
#ratings.to_csv("drive/My Drive/main/ml-1m/ratings_processed.csv")

In [ ]:
# nodes = list(users['user_id']) + list(movies['id'])
edge_list = []
for i in range(len(ratings)):
  edge = (ratings['user_id'][i], ratings['movie_id'][i])
  edge_list.append(edge)
len(edge_list)

1000209

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
G.add_edges_from(edge_list)

In [ ]:
G.number_of_nodes()

9746

In [ ]:
G.number_of_edges()

1000209

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
nx.draw(G)
plt.show()
plt.savefig("movielens_1mil.jpeg")

#colab be like: chal chhainya chhainya chhainya chhainya, chal chhainya chhainya..

Output hidden; open in https://colab.research.google.com to view.

# Trivial Case

### Preparing Positive and Negative Edges

In [ ]:
ratings = pd.read_csv("/content/drive/MyDrive/main/ml-1m/ratings_processed.csv", encoding='ISO-8859-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,u1,m1193,5,978300760
1,u1,m661,3,978302109
2,u1,m914,3,978301968
3,u1,m3408,4,978300275
4,u1,m2355,5,978824291


In [ ]:
ratings = ratings.drop("timestamp", axis=1)
ratings = ratings.drop("rating", axis=1)
ratings.head()

,user_id,movie_id
0,u1,m1193
1,u1,m661
2,u1,m914
3,u1,m3408
4,u1,m2355


In [ ]:
edge_list = []
for i in range(len(ratings)):
  edge = (ratings['user_id'][i], ratings['movie_id'][i])
  edge_list.append(edge)
print("Number of existing edges:",len(edge_list))


Number of existing edges: 1000209


In [ ]:
U = list(ratings["user_id"])
M = list(ratings["movie_id"])
g = nx.Graph()
g.add_nodes_from(U,bipartite=0)
g.add_nodes_from(M,bipartite=1)
g.add_edges_from(edge_list)
print("Number of nodes:", g.number_of_nodes())
print("Number of edges:", g.number_of_edges())
# print("Bipartite sets:",bipartite.sets(g))

In [ ]:
u_vertices = {n for n, d in g.nodes(data=True) if d["bipartite"] == 0}
print(len(u_vertices))
m_vertices = {n for n, d in g.nodes(data=True) if d["bipartite"] == 1}
print(len(m_vertices))

6040
3706


In [ ]:
existing_edges = list(g.edges())
print(len(existing_edges))
positive_edges = pd.DataFrame(existing_edges,columns=['node1', 'node2'])
positive_edges['class']='positive'
positive_edges.head()

1000209


,node1,node2,class
0,u1,m1193,positive
1,u1,m661,positive
2,u1,m914,positive
3,u1,m3408,positive
4,u1,m2355,positive


In [ ]:
non_existing_edges = list(nx.non_edges(g))
print("Total {} non-existing edges present".format(len(non_existing_edges)))

Total 46487176 non-existing edges present


In [ ]:
proper_non_existing_edges= []
for e in non_existing_edges:
  if e[0][0]!=e[1][0]:
    if e[0][0]=='u':
      proper_non_existing_edges.append(e)
    else:
      proper_non_existing_edges.append((e[1],e[0]))
print(len(proper_non_existing_edges))

21384031


In [ ]:
import random 
negative_edges = random.sample(proper_non_existing_edges, 1000000)
negative_edges = pd.DataFrame(negative_edges, columns=["node1","node2"])
negative_edges["class"]="negative"
negative_edges.head()

,node1,node2,class
0,u3414,m1718,negative
1,u2752,m2187,negative
2,u1957,m1416,negative
3,u51,m564,negative
4,u5599,m2527,negative


In [ ]:
print("Entire edge set (positive/existing edges + negative/latent edges):")
edges = pd.concat([positive_edges, negative_edges])
print(len(edges))
edges.head()

Entire edge set (positive/existing edges + negative/latent edges):
2000209


,node1,node2,class
0,u1,m1193,positive
1,u1,m661,positive
2,u1,m914,positive
3,u1,m3408,positive
4,u1,m2355,positive


In [ ]:
# edges.to_csv("drive/My Drive/main/ml-1m/pos_neg_edges.csv")

### Feature Insertion

In [ ]:
edges = pd.read_csv("drive/My Drive/main/ml-1m/pos_neg_edges.csv")
edges = edges.drop(edges.columns[0],axis=1)
edges.head()

,node1,node2,class
0,u1,m1193,positive
1,u1,m661,positive
2,u1,m914,positive
3,u1,m3408,positive
4,u1,m2355,positive


In [ ]:
from math import sqrt, log

def common_neighbors(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    return len(u_nbr.intersection(v_nbr))

def jaccard_coefficient(g,u,v):
	u_nbr = set(g.neighbors(u))
	v_nbr = set(g.neighbors(v))
	return len(u_nbr.intersection(v_nbr))/len(u_nbr.union(v_nbr))

def sorensen_index(g,u,v):
	u_nbr = set(g.neighbors(u))
	v_nbr = set(g.neighbors(v))
	return len(u_nbr.intersection(v_nbr))/(len(u_nbr)+len(v_nbr))

def leicht_holme_nerman(g,u,v):
	u_nbr = set(g.neighbors(u))
	v_nbr = set(g.neighbors(v))
	return len(u_nbr.intersection(v_nbr))/(len(u_nbr)*len(v_nbr))

def salton_cosine_similarity(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    return len(u_nbr.intersection(v_nbr))/sqrt(len(u_nbr)*len(v_nbr))

def hub_promoted_index(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    return len(u_nbr.intersection(v_nbr))/min(len(u_nbr),len(v_nbr))

def hub_depressed_index(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    return len(u_nbr.intersection(v_nbr))/max(len(u_nbr),len(v_nbr))

def preferential_attachment(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    return len(u_nbr)*len(v_nbr)

def resource_allocation(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    cn = u_nbr.intersection(v_nbr)
    ra = 0
    for i in cn:
        ra += 1/float(len(set(g.neighbors(i))))
    return ra

def adamic_adar(g,u,v):
    u_nbr = set(g.neighbors(u))
    v_nbr = set(g.neighbors(v))
    cn = u_nbr.intersection(v_nbr)
    aa = 0
    for i in cn:
        aa += 1/log(len(set(g.neighbors(i))))
    return aa

In [ ]:
import warnings
warnings.filterwarnings("ignore")

feature_set_df = edges.copy()
slength = len(feature_set_df['node1'])
feature_set_df['cn'] = 0.
feature_set_df['jc'] = 0.
feature_set_df['si'] = 0.
feature_set_df['lhn'] = 0.
feature_set_df['scs'] = 0.
feature_set_df['hpi'] = 0.
feature_set_df['hdi'] = 0.
feature_set_df['pa'] = 0.
feature_set_df['aa'] = 0.
feature_set_df['ra'] = 0.
print(feature_set_df.head())
count = 0
for i in range(len(edges)):
    u = edges.iloc[i]['node1']
    v = edges.iloc[i]['node2']
    cn = feature_set_df['cn'][i] = common_neighbors(g,u,v)
    count += 1
    if count%10000 == 0:
        print("Done with {} node-pairs".format(count))
    if cn != 0:
        feature_set_df['jc'][i] = jaccard_coefficient(g,u,v)
        feature_set_df['si'][i] = sorensen_index(g,u,v)
        feature_set_df['lhn'][i] = leicht_holme_nerman(g,u,v)
        feature_set_df['scs'][i] = salton_cosine_similarity(g,u,v)
        feature_set_df['hpi'][i] = hub_promoted_index(g,u,v)
        feature_set_df['hdi'][i] = hub_depressed_index(g,u,v)
        feature_set_df['pa'][i] = float(preferential_attachment(g,u,v))
        feature_set_df['aa'][i] = adamic_adar(g,u,v)
        feature_set_df['ra'][i] = resource_allocation(g,u,v)
        
feature_set_df.head()

In [ ]:
set(g.neighbors("u1"))

{'m1',
 'm1022',
 'm1028',
 'm1029',
 'm1035',
 'm1097',
 'm1193',
 'm1197',
 'm1207',
 'm1246',
 'm1270',
 'm1287',
 'm150',
 'm1545',
 'm1566',
 'm1721',
 'm1836',
 'm1907',
 'm1961',
 'm1962',
 'm2018',
 'm2028',
 'm2294',
 'm2321',
 'm2340',
 'm2355',
 'm2398',
 'm260',
 'm2687',
 'm2692',
 'm2762',
 'm2791',
 'm2797',
 'm2804',
 'm2918',
 'm3105',
 'm3114',
 'm3186',
 'm3408',
 'm48',
 'm527',
 'm531',
 'm588',
 'm594',
 'm595',
 'm608',
 'm661',
 'm720',
 'm745',
 'm783',
 'm914',
 'm919',
 'm938'}

In [ ]:
feature_set_df.to_csv("drive/My Drive/main/ml-1m/edges_feature.csv")